# 데이터 전처리 코드 (rejected cell 제거 및 state 라벨링)

## 코드 사용 설명
miniscope 데이터에서 rejected cell 열 제거 및 state 라벨링하는 코드입니다.

이 코드에서 주로 수정해서 사용할 부분은 바로 아래의 ***Set parameters*** 부분입니다. 해당 부분에 파라미터 설명이 있습니다.
파라미터만 수정하고 Ctrl+A 이후 Ctrl+Enter를 누르시면 모든 행 한 번에 실행 가능합니다.

## 코드 알고리즘 설명 (참고)

### 1. miniscope 데이터 처리
rejected cell을 제거하는 과정에서는 판다스로 데이터를 불러와 첫번째 행에 ' rejected'가 있는 열들을 모두 제거하고 해당 행을 삭제합니다.  
**제가 받았던 파일들에선 'rejected'가 아닌 ' rejected' (제일 앞에 공백문자가 있음)의 형식이었습니다. 만약 해당 형식이 달라진다면 해당 부분만 수정해주시면 됩니다.**  
이후 시간이 있는 행은 데이터 그대로 pandas Dataframe(테이블)의 index로 사용합니다.

### 2. observer data로 state 인코딩(라벨링)
데이터 구조 중 stack 구조를 이용하였습니다. 먼저 옵저버 기록이 이루어진 시간대의 miniscope 테이블 행만 남깁니다. 이후 for문을 이용하여 'State start'가 있는 time bin에서 스택에 push하고, 'State stop'에서는 pop하며, 그 외에는 stack의 가장 위(top)를 이용해 라벨링합니다. 'State point'인 hitass는 스택을 사용하지 않고 라벨링합니다.

반올림을 사용하지 않고 저번에 논의한 대로 절댓값의 차가 가장 작은 time bin에 맞춰 라벨링하였습니다. 그 과정에서 코드가 조금 복잡해졌는데 혹시나 궁금하신 부분 있으시면 연락해주세요.  
**라벨링은 위에서 만든 miniscope table의 마지막 열로 state를 추가하여 하나의 테이블로 통합하였습니다. 최종 결과 테이블이 궁금하시면 Sanity check 부분의 셀을 참고해주세요.**

In [1]:
import numpy as np
import pandas as pd

# *** Set parameters *** (이 부분을 수정해주세요.)

## 파라미터 설명
### LABEL
behavior별로 인코딩할 state를 지정해주는 부분입니다. 

***다만, hitass는 수정하지 말아주세요.***

(이후 rslds 파일에서 hitass를 인식하여 trial을 구분하는 등의 코드에 쓰이고, 해당 파일에서 hitass의 state를 모두 back과 같은 state로 변경합니다. 혹시 수정이 필요하실 경우 연락주세요.)   
(closeopen state는 내부적으로 -2로 자동 지정됩니다. 이는 rslds 파일에서 denoising을 위한 것으로, 이후 다시 지정한 state로 변경됩니다. ***다른 state에는 -2 사용을 피해주세요.***)

### MS_PATH
데이터 가공을 시행할 ***miniscope 데이터의 상대경로***를 넣어주시면 됩니다. 파일명 및 확장자까지 넣어주시는 점 유의해주세요.
### BH_PATH
데이터 가공을 시행할 ***observer 데이터의 상대경로***를 넣어주시면 됩니다. 파일명 및 확장자까지 넣어주시는 점 유의해주세요.
### OUT_PATH
***데이터 가공이 끝난 후의 miniscope + state 테이블을 csv 파일로 저장할 상대경로***를 넣어주시면 됩니다. 파일명 및 확장자까지 넣어주시는 점 유의해주세요.

(제가 받은 파일은 miniscope과 observer파일 각각 csv 확장자, xlsx 확장자였는데 이를 변경할 경우 하단의 Load and preprocess data 부분에서 약간의 수정이 필요합니다. 해당 부분에 기술해 놓았습니다.)

In [2]:
# State label dict
LABEL = {
    'closeopen': 0,
    'search': 1, 
    'fz': 1, 
    'eat': 1, 
    'back': 0, 
    'hitass': -1,   # -1로 고정해주세요.
    'none': 2
}

# Open file path
MS_PATH = "../../data/230406_glp1_k3_day3_k20-2/230719_1trans_glp1_CNMFE_C_4_day3_k-27_day-2.csv"
BH_PATH = "../../data/221117_glp1_ms_k1_day3 - k4-1_2_test - Event Logs/221117_glp1_ms_k4-1_nemo_cond_day0 - k27-day2 - Event Logs.xlsx"
# Output file path
OUT_PATH = "../../data/preprocessed/230406_4.csv"

## Load and preprocess data

바로 밑의 pandas 메소드는 csv 확장자를 불러올 경우 pd.read_csv 메소드로, xlsx 확장자를 불러올 경우 pd.read_excel 메소드를 이용해주시면 됩니다.

In [3]:
data_ms = pd.read_csv(
    MS_PATH, 
    dtype=str 
)
data_bh = pd.read_excel(
    BH_PATH,
    index_col=0,
    usecols=[7,8,11,12]
)

In [4]:
### preprocessing miniscope data

# drop 'rejected' cells
data_ms.drop(
    data_ms.T[(data_ms.loc[0] == ' rejected').values].index.values,
    axis=1,
    inplace = True
)
# drop first row
data_ms = data_ms.iloc[1:, :]
# set data type as float
data_ms = data_ms.astype(float)
# set time bins to index
col = data_ms.columns[0]
data_ms = data_ms.set_index(col, drop=True)

In [5]:
data_ms

,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,...,C43,C45,C46,C47,C48,C49,C52,C55,C59,C67
,,,,,,,,,,,,,,,,,,,,,
0.000000,0.000000,1.018762,0.000000,0.122158,0.000000,0.000000,0.000000,4.292403,0.028805,0.000000,...,0.000000,4.531135,0.000000,0.000000,1.221227,0.000000,1.150110,0.000000,0.077064,0.000000
0.099922,0.000000,1.014120,0.000000,0.121885,0.000000,0.000000,0.000000,4.278659,0.028707,0.000000,...,0.000000,4.494563,0.000000,0.000000,1.202029,0.000000,1.132371,0.000000,0.076455,0.000000
0.199844,0.000000,1.009500,0.000000,0.121612,0.000000,0.000000,0.000000,4.264959,0.028608,0.000000,...,0.000000,4.458286,0.000000,0.000000,1.183133,0.000000,1.114906,0.000000,0.075851,0.000000
0.299766,0.000000,1.004900,0.000000,0.121340,0.000000,0.000000,0.000000,4.251303,0.028510,0.000000,...,0.000000,4.422302,0.000000,0.000000,1.164534,0.000000,1.097710,0.000000,0.075252,0.000000
0.399688,0.000000,1.000322,0.000000,0.121069,0.000000,0.000000,0.000000,4.237690,0.028412,0.000000,...,0.000000,4.386609,0.000000,0.000000,1.146227,0.000000,1.080780,0.000000,0.074657,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4293.648340,0.374264,1.550367,0.452090,0.504771,0.000139,0.007565,0.100961,1.190910,0.017127,0.365938,...,1.250209,0.000004,0.004241,0.372968,0.281382,0.000003,0.220482,0.006802,0.000805,0.028812
4293.748262,0.372758,1.543303,0.449843,0.503641,0.000139,0.007552,0.100400,1.187097,0.017068,0.363618,...,1.242070,0.000004,0.004174,0.368176,0.276958,0.000003,0.217082,0.006698,0.000799,0.028255
4293.848184,0.371257,1.536272,0.447607,0.502514,0.000138,1.646924,0.099842,1.183296,0.017009,0.361313,...,1.233983,0.000004,0.004109,0.363446,0.272605,0.000003,0.213733,0.006594,0.000792,0.027709


In [6]:
### encoding states

# data_bh.index = np.round(data_bh.index, 1)
data_ms['state'] = np.NaN

# create a stack
label_stack = [LABEL['none']]

# change data_bh indices
index_list = [np.argmin(abs(data_ms.index - index)) for index in data_bh.index.values]
data_bh.set_index(data_ms.iloc[index_list].index, inplace=True)
bh_index = data_bh.index

# 옵저버 기록 전후 행 삭제
data_ms = data_ms.loc[bh_index[0]:bh_index[-1], :]

# normalize miniscope data using 'closeopen' state (Denoising)
LABEL['closeopen'] = -2   # -2로 고정

for index, row in data_ms.iterrows():
    # iteration 마지막에 stack에서 top 꺼낼지 여부
    top_stack = True
    # if start/stop time bins
    if index in bh_index:
        bh_row = data_bh.loc[index]
        event_type = bh_row['Event_Type']
        
        ### 같은 time index에 두 행 있을 경우 처리
        if type(event_type) != str:
            for _, r in bh_row.iterrows():
                e = r.Event_Type
                if e == 'State start':
                    label_stack.append(LABEL[r.Behavior])     
                elif e == 'State stop':
                    row['state'] = label_stack.pop()
                    top_stack = False
                elif e == 'State point':
                    row['state'] = LABEL[r.Behavior]
                    top_stack = False
            
        # if event type is start time, append state to stack
        elif event_type == 'State start':
            label_stack.append(LABEL[bh_row.Behavior])     
            
        # if it's stop time, pop state from stack
        elif event_type == 'State stop':
            row['state'] = label_stack.pop()
            top_stack = False        
            
        # if it's state point (hitass), set state directly without using stack
        elif event_type == 'State point':
            row['state'] = LABEL[bh_row.Behavior]
            top_stack = False
            
    if top_stack:
        row['state'] = label_stack[-1] 

### Sanity check
데이터 가공의 결과물을 확인하고, state가 -1(hitass)인 행의 개수를 확인하여 trial 수를 확인하는 등 기본적인 점검을 하는 셀입니다.

In [7]:
data_ms

,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,...,C45,C46,C47,C48,C49,C52,C55,C59,C67,state
,,,,,,,,,,,,,,,,,,,,,
0.299766,0.000000,1.004900,0.000000,0.121340,0.000000,0.000000,0.000000,4.251303,0.028510,0.000000,...,4.422302,0.000000,0.000000,1.164534e+00,0.000000,1.097710e+00,0.000000,7.525195e-02,0.000000e+00,-2.0
0.399688,0.000000,1.000322,0.000000,0.121069,0.000000,0.000000,0.000000,4.237690,0.028412,0.000000,...,4.386609,0.000000,0.000000,1.146227e+00,0.000000,1.080780e+00,0.000000,7.465744e-02,0.000000e+00,-2.0
0.499610,0.000000,0.995764,0.000000,0.120798,0.000000,0.000000,0.000000,4.224121,0.028315,0.000000,...,4.351203,0.000000,0.000000,1.128209e+00,0.000000,1.064111e+00,0.000000,7.406762e-02,0.000000e+00,-2.0
0.599532,0.000000,0.991227,0.000000,0.120528,0.000000,0.000000,0.000000,4.210596,0.028217,0.000000,...,4.316083,0.000000,0.000000,1.110473e+00,0.000000,1.047698e+00,0.000000,7.348247e-02,0.000000e+00,-2.0
0.699454,0.000000,0.986711,0.000000,0.120258,0.000000,0.000000,0.000000,4.197114,0.028121,0.000000,...,4.281247,0.000000,0.000000,1.093017e+00,0.000000,1.031539e+00,0.000000,7.290193e-02,0.000000e+00,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4144.364872,0.003967,0.000607,1.059915,0.605716,0.015498,0.103110,2.839045,1.808121,0.302412,0.048233,...,0.709361,2.038953,1.107436,7.103171e-14,0.055774,4.072647e-29,0.277819,1.110280e-15,9.406514e-13,0.0
4144.464794,0.003951,0.000604,1.054647,0.604361,0.015449,0.102930,2.823268,1.802332,0.301374,0.047927,...,0.703636,2.006927,1.093209,6.991509e-14,0.055200,4.009832e-29,0.273536,1.101509e-15,9.224778e-13,0.0
4144.564716,0.003935,0.000602,1.049405,0.603009,0.015401,0.102750,2.807579,1.796561,0.300339,0.047623,...,0.697957,1.975405,1.079165,6.881602e-14,0.054633,3.947987e-29,0.269318,1.092806e-15,9.046553e-13,0.0


In [8]:
data_ms.loc[data_ms['state'] == -1.0]

,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,...,C45,C46,C47,C48,C49,C52,C55,C59,C67,state
,,,,,,,,,,,,,,,,,,,,,
169.267868,3.721164,13.083140,10.112360,10.118780,0.000000,6.607237,4.243337,14.287720,8.105037,4.028849,...,4.426981e+00,0.006144,0.799109,1.923533e-04,7.918391e-05,4.220439e-12,0.000000e+00,4.303585e-01,1.470376e-04,-1.0
319.550556,3.853272,6.982576,2.170093,3.606022,3.784340,3.726957,1.833203,3.857992,3.457216,1.198441,...,4.644753e+00,0.119899,0.727332,3.459271e+00,7.661473e-01,2.968758e-22,3.509606e-03,1.446471e-01,1.187748e-04,-1.0
467.235272,2.205784,6.572596,1.720063,3.878937,1.897898,3.773801,3.551082,6.426887,3.475754,3.092478,...,1.495721e+00,3.533895,2.980419,1.568626e+00,2.158342e+00,3.052615e-08,1.389355e+00,5.316281e-02,3.012591e-01,-1.0
679.369678,2.842107,8.271362,1.946135,2.355953,2.163526,2.369378,7.498707,6.939163,0.529128,0.981577,...,2.988751e+00,0.590843,3.755848,1.706254e+00,4.225391e-01,8.556801e-07,3.660473e-03,4.667234e-02,4.455842e+00,-1.0
842.742148,1.209362,1.849466,1.001514,0.185040,5.299485,0.135855,5.996070,1.288252,0.015146,0.578094,...,3.220889e+00,0.449709,3.806638,3.223393e-03,1.503621e-02,7.856161e-18,1.784216e-03,2.065265e-01,5.274012e+00,-1.0
1020.503386,1.070282,1.617127,2.111434,0.428950,2.871437,1.189188,2.402688,3.437752,0.166652,0.244725,...,1.421982e-02,1.934433,0.069415,3.938586e-01,1.031714e+00,5.448581e-03,3.653418e-03,1.299573e-01,3.108761e+00,-1.0
1201.062440,0.238255,7.647520,2.072521,1.115045,0.782000,2.396688,3.829149,3.151658,1.483492,3.227833,...,1.831842e+00,1.300144,0.228753,8.223878e-01,5.605802e-01,4.995370e-06,1.314691e-02,2.407708e-02,1.022751e+01,-1.0
1352.544192,0.397333,0.007533,1.981020,4.663987,1.829394,2.191005,3.707611,4.623220,1.277296,0.977530,...,8.458515e-06,0.632961,1.247569,7.355455e-02,1.963358e-06,3.135950e-02,4.111825e-03,1.443711e-07,3.529347e-07,-1.0
1517.515414,0.809551,0.163908,1.979128,0.115541,2.986897,1.614633,0.071292,0.650656,0.004379,1.224479,...,6.074753e-01,1.059685,4.646237,8.473888e-02,8.515344e-01,4.364849e-11,1.143396e-01,2.967057e-13,3.623162e-21,-1.0


## Save data to csv
바로 위의 테이블 결과에서 C00, C01 등의 셀 번호 (첫번째 행)는 제외하고 저장합니다. 이를 원치 않을 경우 header=True 로 변경해주세요.

In [9]:
data_ms.to_csv(OUT_PATH, header=False)